#  Clase 18 - SQL

### Anuncios

- Jueves no hay clases

### Hoy

- Conexion directa vs indirecta
- Uso de JOIN
- Uso de funcion “CASE”

#### Opciones de consulta de datos: Método "directo"

In [36]:
import sqlite3

conn = sqlite3.connect('notes.db')

conn

In [37]:
cursor = conn.cursor()
cursor

Un cursor es un objeto que permite recorrer los registros de una tabla. Se puede recorrer de forma secuencial o aleatoria. En este caso, se recorre de forma secuencial.

In [38]:
query = 'SELECT * FROM NOTAS'

response = cursor.execute(query)

response

In [39]:
# data = response.fetchone()
# data

In [40]:
data = response.fetchall()
data

[(0, 'A', 'c1', 0.1, 6.0),
 (1, 'A', 'p1', 0.2, 5.0),
 (2, 'A', 'p2', 0.4, 7.0),
 (3, 'B', 'c1', 0.05, 4.0),
 (4, 'B', 'p1', 0.3, 5.0),
 (5, 'B', 'c2', 0.05, 6.0),
 (6, 'C', 'p1', 0.25, 4.0),
 (7, 'C', 'p2', 0.25, 5.0),
 (8, 'D', 'p1', 0.3, 6.5)]

In [41]:
type(data)

list

#### Opciones de consulta de datos: Método "indirecto"

In [42]:
import pandas as pd
import sqlite3

conn = sqlite3.connect('notes.db')

query = 'SELECT * FROM NOTAS'

df = pd.read_sql(query, conn)

df



,index,Curso,Evaluacion,Ponderacion,Nota
0,0,A,c1,0.10,6.0
1,1,A,p1,0.20,5.0
2,2,A,p2,0.40,7.0
3,3,B,c1,0.05,4.0
4,4,B,p1,0.30,5.0
5,5,B,c2,0.05,6.0
6,6,C,p1,0.25,4.0
7,7,C,p2,0.25,5.0
8,8,D,p1,0.30,6.5


In [43]:
import sqlite3
import pandas

#metodo indirecto, usando pandas

conn = sqlite3.connect('data_bb.db')

conn

In [44]:
query = '''
    SELECT TICKER, COUNT(VALOR) AS N_FECHAS
    FROM PRECIOS
    GROUP BY TICKER
    ORDER BY N_FECHAS
'''

df = pandas.read_sql(query, conn)

df

DatabaseError: Execution failed on sql '
    SELECT TICKER, COUNT(VALOR) AS N_FECHAS
    FROM PRECIOS
    GROUP BY TICKER
    ORDER BY N_FECHAS
': no such table: PRECIOS

### VEAMOS SI VALOR TIENE VALORES NULOS

In [ ]:
query = '''
    SELECT *
    FROM PRECIOS
    WHERE VALOR IS NULL
'''

df = pandas.read_sql(query, conn)

df

,index,fecha,valor,ticker


#### BUSQUEMOS EN QUE FECHAS EL IPSA NO TIENE PRECIOS

In [ ]:
query = '''
    WITH FECHAS AS (
    SELECT DISTINCT FECHA
    FROM PRECIOS
    ),
    PRECIOS_IPSA AS (
    SELECT *
    FROM PRECIOS
    WHERE TICKER = 'IPSA Index'
    )
    
    SELECT *
    FROM FECHAS LEFT JOIN PRECIOS_IPSA USING (FECHA)
    WHERE VALOR IS NULL
'''

df = pandas.read_sql(query, conn)

df

,FECHA,index,valor,ticker
0,2011-04-22,None,None,None
1,2011-08-15,None,None,None
2,2012-08-15,None,None,None


In [ ]:
query = '''
    SELECT *
    FROM PRECIOS
    WHERE TICKER = 'IPSA Index'
'''

df = pandas.read_sql(query, conn)

df

,index,fecha,valor,ticker
0,2,2010-01-04,3620.70,IPSA Index
1,3,2010-01-05,3624.10,IPSA Index
2,4,2010-01-06,3639.96,IPSA Index
3,5,2010-01-07,3681.06,IPSA Index
4,6,2010-01-08,3726.50,IPSA Index
...,...,...,...,...
3436,3438,2023-10-17,5890.74,IPSA Index
3437,3439,2023-10-18,5796.44,IPSA Index
3438,3440,2023-10-19,5713.44,IPSA Index
3439,3441,2023-10-20,5631.33,IPSA Index


### ejemplo clase pasada

In [ ]:
import pandas as pd
import sqlite3

conn = sqlite3.connect('notas.db')

#en pandas no es necesario crear un objeto cursor

query = '''
    -- AGREGAMOS MAXIMO
    WITH INTERMEDIA AS ( 
        SELECT CURSO, 
            SUM(NOTA*PONDERACION)/SUM(PONDERACION) AS NOTA_ACTUAL,
            SUM(PONDERACION) AS PORC_ACUM
        FROM NOTAS 
        GROUP BY CURSO
    )
    SELECT CURSO, NOTA_ACTUAL, PORC_ACUM,
        MAX((? - NOTA_ACTUAL*PORC_ACUM) / (1-PORC_ACUM), 1) AS NECESARIA
    FROM INTERMEDIA
'''

df = pd.read_sql(query, conn, params=(6,))

df

,CURSO,NOTA_ACTUAL,PORC_ACUM,NECESARIA
0,A,6.285714,0.7,5.333333
1,B,5.000000,0.4,6.666667
2,C,4.500000,0.5,7.500000
3,D,6.500000,0.3,5.785714
